In [ ]:
import json
import base64
import random
import requests
import numpy as np
from PIL import Image
from datetime import datetime
from cryptography.fernet import Fernet

def generate_key():
    return Fernet.generate_key()

# Encrypt a message using symmetric encryption
def encrypt_message(message, key):
    cipher = Fernet(key)
    return cipher.encrypt(message.encode())

def decrypt_message(ciphertext, key):
    cipher = Fernet(key)
    return cipher.decrypt(ciphertext).decode()

def get_current_location():
    response = requests.get('https://ipinfo.io/json')
    data = response.json()
    city = data.get('city', '')
    region = data.get('region', '')
    country = data.get('country', '')
    loc = data.get('loc', '')  # latitude,longitude
    return {
        'city': city,
        'region': region,
        'country': country,
        'coordinates': loc
    }

## Spread Spectrum Encoder

In [18]:
"""
Encodes a message into an image using spread spectrum technique.

This function embeds a given message into an image by converting the message
into a bit sequence and spreading it across the image's pixel data using a
pseudorandom noise sequence generated from a key. The resulting watermarked
image is saved to the specified output path.

Parameters:
    image_path (str): Path to the input image file.
    message (str): The message to encode into the image.
    key (int): Seed for generating the pseudorandom noise sequence.
    output_path (str): Path to save the watermarked output image.

Raises:
    ValueError: If the image is too small to encode the message.
"""
def ss_encode(image_path, message, key, output_path):
    # Load image and flatten to 1D array
    img = Image.open(image_path).convert('L')
    arr = np.array(img).astype(np.float64)
    flat = arr.flatten()
    length = flat.size

    # Convert message to bits
    message_bits = np.array([int(b) for c in message for b in f'{ord(c):08b}'])

    # Repeat message bits to fill the image length
    spread_seq = np.tile(message_bits, int(np.ceil(length / len(message_bits))))[:length]

    # Generate pseudorandom noise using key
    random.seed(key)
    pn_sequence = np.array([random.choice([-1, 1]) for _ in range(length)])

    # Embed: add small amplitude of spread message*pn_sequence
    amplitude = 2  # Controls strength (in grayscale 0-255)
    watermarked = flat + amplitude * (2 * spread_seq - 1) * pn_sequence
    watermarked = np.clip(watermarked, 0, 255).astype(np.uint8)
    out_img = Image.fromarray(watermarked.reshape(arr.shape))
    out_img.save(output_path)

## Spread Spectrum Decoder

In [19]:
"""
Decodes a message from an image using the spread spectrum technique.

This function extracts an embedded message from an image by correlating the
image's pixel data with a pseudorandom noise sequence generated from a key.
The message is reconstructed from the correlated bit sequence.

Parameters:
    image_path (str): Path to the input image file.
    message_length (int): Length of the message to decode.
    key (int): Seed for generating the pseudorandom noise sequence.

Returns:
    str: The decoded message from the image.
"""
def ss_decode(image_path, message_length, key):
    img = Image.open(image_path).convert('L')
    arr = np.array(img).astype(np.float64)
    flat = arr.flatten()
    length = flat.size

    spread_length = length // (message_length * 8)
    indices = np.arange(0, length, spread_length)[:message_length * 8]

    random.seed(key)
    pn_sequence = np.array([random.choice([-1, 1]) for _ in range(length)])

    # Correlate over each bit's region
    bits = []
    for i, idx in enumerate(indices):
        region = flat[idx:idx+spread_length]
        pn = pn_sequence[idx:idx+spread_length]
        if region.size == 0:
            continue
        bit_val = np.sign(np.sum(region * pn))
        bits.append(1 if bit_val > 0 else 0)
    # Convert bits to chars
    chars = [chr(int(''.join(map(str, bits[i:i+8])), 2)) for i in range(0, len(bits), 8)]
    message = ''.join(chars)
    return message

## Usage example

In [20]:
inputPath = "assets/images/raw.png"
outputPath = "assets/images/ss-encoded.png"

secret_data = {
    "message": "Very Secret Message",
    "date": datetime.now().isoformat(),
    "location": get_current_location()
}

secret_message = json.dumps(secret_data, indent=4)

key = generate_key()
print("Generated encryption key (store this securely):")
print(key.decode())

# To encode:
ss_encode(inputPath, secret_message, key, outputPath)

# To decode:
message = ss_decode(outputPath, len(secret_message), key)
print("\nDecoded message:")
print(secret_message)

Generated encryption key (store this securely):
jA3mhjclb9INJ2bi4CSxwyRfZrPLdsnUDzVgmSQiGdQ=

Decoded message:
{
    "message": "Very Secret Message",
    "date": "2025-06-15T09:19:57.821221",
    "location": {
        "city": "Plessisville",
        "region": "Quebec",
        "country": "CA",
        "coordinates": "46.2186,-71.7620"
    }
}
